In [8]:
import sys, os, pickle
sys.path.append('/home/om2382/mft-theory/')
from cluster import *
from utils import *
import h5py
import matplotlib.pyplot as plt

In [9]:
### --- SET UP ALL CONFIGS --- ###
from itertools import product
n_seeds = 1
macro_configs = config_generator(i_chunk=list(range(9)))

micro_configs = tuple(product(macro_configs, list(range(n_seeds))))
prototype = False

### --- SELECT PARTICULAR CONFIG --- ###
try:
    i_job = int(os.environ['SLURM_ARRAY_TASK_ID']) - 1
except KeyError:
    i_job = 0
    prototype = True
params, i_seed = micro_configs[i_job]
i_config = i_job//n_seeds

new_random_seed_per_condition = True
if new_random_seed_per_condition:
    np.random.seed(i_job)
else: #Match random seeds across conditions
    np.random.seed(i_seed)

In [ ]:
path = '/home/om2382/datasets/20210305_both_6mm_FOV_50_550um_depth_350mW_60min_no_stim_M1.h5?download=1'
with h5py.File(path, "r") as f:
    T, N = f["T_all"].shape
if params['i_chunk'] == 0:
    t_path = '/home/om2382/datasets/20210305_both_6mm_FOV_50_550um_depth_350mW_60min_no_stim_M1_transpose.h5'
    with h5py.File(t_path, 'w') as f:
        f.create_dataset("T_all_transpose", data=np.zeros((N, T)))

In [14]:
if params['i_chunk'] > 0:
    path = '/home/om2382/datasets/20210305_both_6mm_FOV_50_550um_depth_350mW_60min_no_stim_M1.h5?download=1'
    with h5py.File(path, "r") as f:
        T, N = f["T_all"].shape
        transposed_data = np.zeros((N, 1000))
        start = (params['i_chunk'] - 1)*1000
        end = np.minimum(T, params['i_chunk']*1000)
        for t in range(start, end):
            if t%100 == 0:
                print(t)
            transposed_data[:,t] = f["T_all"][t,:]
    t_path = '/home/om2382/datasets/20210305_both_6mm_FOV_50_550um_depth_350mW_60min_no_stim_M1_transpose.h5'
    with h5py.File(t_path, 'w') as f:
        f["T_all_transpose"][:,start:end] = transposed_data

In [ ]:
### --- SAVE RESULTS -- ###
result = {'sim': None, 'i_seed': i_seed, 'config': params,
          'i_config': i_config, 'i_job': i_job}
try:
    result['processed_data'] = processed_data
except NameError:
    pass
    
try:
    save_dir = os.environ['SAVEDIR']
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_path = os.path.join(save_dir, 'result_{}'.format(i_job))

    with open(save_path, 'wb') as f:
        pickle.dump(result, f)
except KeyError:
    pass

In [5]:
###Truncate file above
file_name = 'vaziri_transpose_2'
job_name = 'vaziri_t'
project_dir = '/home/om2382/low-rank-dims/'
main_script_path = os.path.join(project_dir, 'cluster_main_scripts', job_name + '.py')
get_ipython().run_cell_magic('javascript', '', 'IPython.notebook.save_notebook()')
get_ipython().system('jupyter nbconvert --to script --no-prompt {}.ipynb'.format(file_name))
get_ipython().system('awk "/###Truncate/ {{exit}} {{print}}" {}.py'.format(file_name))
get_ipython().system('sed -i "/###Truncate/Q" {}.py'.format(file_name))
get_ipython().system('mv {}.py {}'.format(file_name, main_script_path))

<IPython.core.display.Javascript object>

[NbConvertApp] Converting notebook vaziri_transpose_2.ipynb to script
[NbConvertApp] Writing 3572 bytes to vaziri_transpose_2.py
awk: cmd. line:1: /###Truncate/ <IPython.core.autocall.ZMQExitAutocall object at 0x2b8d93544b10> <built-in function print>
awk: cmd. line:1:                       ^ syntax error
awk: cmd. line:1: /###Truncate/ <IPython.core.autocall.ZMQExitAutocall object at 0x2b8d93544b10> <built-in function print>
awk: cmd. line:1:                                                                                ^ syntax error


In [6]:
###Submit job to cluster
n_jobs = len(micro_configs)
write_job_file(job_name, py_file_name='{}.py'.format(job_name), mem=16, n_hours=1, n_gpus=1)
job_script_path = os.path.join(project_dir, 'job_scripts', job_name + '.s')
submit_job(job_script_path, n_jobs, execute=False)

rm: cannot remove ‘/home/om2382/low-rank-dims/results/misc/vaziri_t/result_*’: No such file or directory
sending incremental file list
mft-theory/
mft-theory/.DS_Store
mft-theory/.gitignore
mft-theory/README.md
mft-theory/__init__.py
mft-theory/jupyter_notebook.py
mft-theory/main.ipynb
mft-theory/.idea/
mft-theory/.idea/mft-theory.iml
mft-theory/.idea/misc.xml
mft-theory/.idea/modules.xml
mft-theory/.idea/vanilla-rtrl.iml
mft-theory/.idea/vcs.xml
mft-theory/.idea/workspace.xml
mft-theory/.idea/codeStyles/
mft-theory/.idea/codeStyles/codeStyleConfig.xml
mft-theory/.ipynb_checkpoints/
mft-theory/.ipynb_checkpoints/main-checkpoint.ipynb
mft-theory/cluster/
mft-theory/cluster/__init__.py
mft-theory/cluster/close_jupyter_notebook.py
mft-theory/cluster/process_results.py
mft-theory/cluster/start_jupyter_notebook.py
mft-theory/cluster/submit_jobs.py
mft-theory/cluster/sync_cluster.py
mft-theory/core/
mft-theory/core/Simulation.py
mft-theory/core/Time_Cts_RNN.py
mft-theory/core/Torch_Sim.py
mf

In [14]:
###Get job status
get_ipython().system('squeue -u om2382')

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           3047004    lkumar  jupyter   om2382  R      56:28      1 ax16


In [15]:
project_dir = '/home/om2382/low-rank-dims/'
job_name = 'n=4k_usv_spectra'
job_script_path = os.path.join(project_dir, 'job_scripts', job_name + '.s')
configs_array, results_array, key_order, sim_dict = unpack_processed_data(job_script_path)